In [6]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.ndimage import gaussian_filter1d
from scipy.spatial.transform import Rotation as R

pd.set_option('display.max_columns', None)

In [2]:
# Quaternion to Euler Angles
def q2e(row, rotation_order='XYZ', degrees=True):
    result = {}
    num_joints = len([col for col in row.index if col.endswith('x')])  # Joint 수 계산

    for jointNum in range(num_joints):
        # 퀘터니언 추출
        x = row[f"b{jointNum}x"]
        y = row[f"b{jointNum}y"]
        z = row[f"b{jointNum}z"]
        w = row[f"b{jointNum}w"]

        # scipy를 사용한 오일러 각도 변환
        r = R.from_quat([x, y, z, w])  # [x, y, z, w] 순서로 입력
        euler_angles = r.as_euler(rotation_order.lower(), degrees=degrees)  # 회전 순서와 단위 설정

        # 결과 저장
        result.update({
            f'b{jointNum}_euler_x': euler_angles[0],  # 첫 번째 축(roll)
            f'b{jointNum}_euler_y': euler_angles[1],  # 두 번째 축(pitch)
            f'b{jointNum}_euler_z': euler_angles[2],  # 세 번째 축(yaw)
        })

    return pd.Series(result)

In [21]:
# txt file read
column_names = ["b0x", "b0y", "b0z", "b0w", 
                "b1x", "b1y", "b1z", "b1w", 
                "b2x", "b2y", "b2z", "b2w", 
                "b3x", "b3y", "b3z", "b3w", 
                "b4x", "b4y", "b4z", "b4w", 
                "b5x", "b5y", "b5z", "b5w", 
                "b6x", "b6y", "b6z", "b6w", 
                "b7x", "b7y", "b7z", "b7w", 
                "b8x", "b8y", "b8z", "b8w", 
                "b9x", "b9y", "b9z", "b9w"]

root = "C:/Users/Velab/Desktop/mocap/문워크2/FB-Raw-Data-000-165643.txt"

In [22]:
df = pd.read_csv(root, sep="\t", skiprows=2, header=None, names=column_names)
df.head()

,b0x,b0y,b0z,b0w,b1x,b1y,b1z,b1w,b2x,b2y,b2z,b2w,b3x,b3y,b3z,b3w,b4x,b4y,b4z,b4w,b5x,b5y,b5z,b5w,b6x,b6y,b6z,b6w,b7x,b7y,b7z,b7w,b8x,b8y,b8z,b8w,b9x,b9y,b9z,b9w
0,0.457543,-0.498717,-0.340825,-0.652514,0.554261,0.436607,-0.515813,0.485907,-0.584630,0.210514,0.759044,0.194280,0.459707,0.508749,-0.415696,0.597529,0.733498,-0.394475,-0.538577,0.127693,0.564041,0.383889,-0.647388,0.339670,-0.503184,-0.470129,0.574089,-0.442952,0.571977,0.487863,0.056058,0.657031,-0.997148,0.003653,-0.005284,0.075194,0.999950,0.007780,-0.002968,0.005457
1,0.456966,-0.499002,-0.340845,-0.652690,0.554813,0.435613,-0.517059,0.484844,-0.584634,0.210961,0.758858,0.194509,0.459277,0.509037,-0.415833,0.597519,0.733661,-0.394335,-0.538469,0.127643,0.563635,0.384282,-0.647462,0.339757,-0.503184,-0.470129,0.574089,-0.442952,0.572062,0.487796,0.056018,0.657010,-0.997145,0.003659,-0.005253,0.075235,0.999950,0.007787,-0.002977,0.005466
2,0.456780,-0.499130,-0.341131,-0.652572,0.554408,0.435978,-0.517533,0.484474,-0.584268,0.211290,0.759054,0.194488,0.459163,0.509074,-0.415963,0.597485,0.733698,-0.394336,-0.538445,0.127527,0.563765,0.384007,-0.647687,0.339424,-0.503184,-0.470129,0.574089,-0.442952,0.572087,0.487832,0.055853,0.656975,-0.997143,0.003654,-0.005266,0.075258,0.999950,0.007756,-0.002994,0.005478
3,0.456556,-0.499123,-0.341785,-0.652393,0.554519,0.435634,-0.518274,0.483864,-0.584513,0.211218,0.759034,0.193903,0.459786,0.508691,-0.416853,0.596711,0.733910,-0.394797,-0.537899,0.127183,0.563856,0.383817,-0.647963,0.338960,-0.503184,-0.470129,0.574089,-0.442952,0.572036,0.487964,0.055714,0.656934,-0.997144,0.003672,-0.005252,0.075253,0.999951,0.007754,-0.002985,0.005462
4,0.456872,-0.497971,-0.342706,-0.652569,0.554425,0.435664,-0.518549,0.483649,-0.585208,0.210426,0.759322,0.191526,0.460549,0.508432,-0.418376,0.595276,0.734360,-0.395579,-0.536873,0.126492,0.563842,0.383983,-0.648031,0.338667,-0.503184,-0.470129,0.574089,-0.442952,0.571765,0.488562,0.055513,0.656742,-0.997142,0.003679,-0.005246,0.075283,0.999951,0.007739,-0.002996,0.005451


In [23]:
# Quaternion to Euler
result_df = df.apply(q2e, axis=1)
result_df.head()

,b0_euler_x,b0_euler_y,b0_euler_z,b1_euler_x,b1_euler_y,b1_euler_z,b2_euler_x,b2_euler_y,b2_euler_z,b3_euler_x,b3_euler_y,b3_euler_z,b4_euler_x,b4_euler_y,b4_euler_z,b5_euler_x,b5_euler_y,b5_euler_z,b6_euler_x,b6_euler_y,b6_euler_z,b7_euler_x,b7_euler_y,b7_euler_z,b8_euler_x,b8_euler_y,b8_euler_z,b9_euler_x,b9_euler_y,b9_euler_z
0,-71.936156,74.306705,-2.454258,87.185171,84.932455,-11.285381,22.083026,75.769876,168.552216,64.723776,81.964294,-11.985693,122.314706,43.578405,-81.374330,-58.796695,82.349230,-177.098871,-61.255331,83.844152,-160.687076,99.182141,35.236652,50.666969,-171.372758,-0.572322,-0.462935,179.377318,0.344965,0.889677
1,-71.784840,74.343291,-2.361866,86.829747,84.971318,-11.862387,22.188428,75.783660,168.600478,64.489090,82.006378,-12.187704,122.343260,43.585725,-81.337015,-58.847718,82.303143,-177.102602,-61.255331,83.844152,-160.687076,99.186100,35.231547,50.657270,-171.368048,-0.568648,-0.463423,179.376324,0.346021,0.890464
2,-71.719893,74.382596,-2.290662,86.596385,85.061965,-12.099702,22.298933,75.735267,168.682893,64.380214,82.020277,-12.297449,122.358207,43.593328,-81.331928,-58.823146,82.294122,-177.135204,-61.255331,83.844152,-160.687076,99.196807,35.245512,50.652243,-171.365410,-0.570187,-0.463011,179.374891,0.347891,0.886924
3,-71.559850,74.436984,-2.093049,86.324881,85.109987,-12.474922,22.445709,75.752720,168.886341,64.417634,82.058329,-12.414713,122.464702,43.560204,-81.300943,-58.885981,82.270402,-177.258922,-61.255331,83.844152,-160.687076,99.209137,35.266431,50.660625,-171.365953,-0.568480,-0.464855,179.376719,0.346946,0.886640
4,-71.242471,74.379357,-1.647047,86.194013,85.143833,-12.626735,22.839397,75.772405,169.550954,64.433693,82.171227,-12.629615,122.672046,43.506812,-81.231177,-59.100007,82.246932,-177.484052,-61.255331,83.844152,-160.687076,99.255086,35.326662,50.727652,-171.362564,-0.567755,-0.465687,179.377990,0.348179,0.884971


In [24]:
# Dataframe save
np.savetxt('C:/Users/Velab/Desktop/mocap/result/moonwalk02_01.csv', result_df, delimiter=",");